In [1]:
# 导入
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体为黑体
plt.rcParams['axes.unicode_minus'] = False  # 正确显示负号
import warnings
warnings.filterwarnings("ignore")

In [2]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, recall_score, f1_score
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
from sklearn.ensemble import VotingClassifier
import torch

In [3]:

# 读取数据
df = pd.read_csv("./data/error/2001.csv", index_col=0) #M101 test ./data/error/1001.csv
df

,日期,时间,生产线编号,物料推送气缸推送状态,物料推送气缸收回状态,物料推送数,物料待抓取数,放置容器数,容器上传检测数,填装检测数,...,不合格数,物料推送装置故障1001,物料检测装置故障2001,填装装置检测故障4001,填装装置定位故障4002,填装装置填装故障4003,加盖装置定位故障5001,加盖装置加盖故障5002,拧盖装置定位故障6001,拧盖装置拧盖故障6002
1169722,57,11969,M101,1,0,1764,1758,598,597,586,...,9,0,2001,0,0,0,0,0,0,0
1169723,57,11970,M101,0,1,1764,1758,598,597,587,...,9,0,2001,0,0,0,0,0,0,0
1169724,57,11971,M101,0,1,1764,1758,598,597,587,...,9,0,2001,0,0,0,0,0,0,0
1169725,57,11972,M101,0,1,1764,1758,598,597,587,...,9,0,2001,0,0,0,0,0,0,0
1169726,57,11973,M101,0,1,1764,1758,598,597,587,...,9,0,2001,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509769,73,4911,M110,0,1,768,768,258,257,256,...,0,0,0,0,0,0,0,0,0,0
549598,25,28486,M110,0,1,4443,4443,1482,1481,1480,...,0,0,0,0,0,0,0,0,0,0
5743966,277,11406,M110,0,1,1768,1767,591,590,589,...,0,0,0,0,0,0,0,0,0,0
2408728,116,7095,M110,0,1,1107,1107,371,370,369,...,0,0,0,0,0,0,0,0,0,0


In [4]:
n = len(df.columns)
# 选择倒数前九列
dfx = df.iloc[:, :n-9]

# 进行独热编码
dfx = pd.get_dummies(dfx, columns=['生产线编号'])
dfx

,日期,时间,物料推送气缸推送状态,物料推送气缸收回状态,物料推送数,物料待抓取数,放置容器数,容器上传检测数,填装检测数,填装定位器固定状态,...,生产线编号_M101,生产线编号_M102,生产线编号_M103,生产线编号_M104,生产线编号_M105,生产线编号_M106,生产线编号_M107,生产线编号_M108,生产线编号_M109,生产线编号_M110
1169722,57,11969,1,0,1764,1758,598,597,586,0,...,True,False,False,False,False,False,False,False,False,False
1169723,57,11970,0,1,1764,1758,598,597,587,1,...,True,False,False,False,False,False,False,False,False,False
1169724,57,11971,0,1,1764,1758,598,597,587,1,...,True,False,False,False,False,False,False,False,False,False
1169725,57,11972,0,1,1764,1758,598,597,587,1,...,True,False,False,False,False,False,False,False,False,False
1169726,57,11973,0,1,1764,1758,598,597,587,1,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509769,73,4911,0,1,768,768,258,257,256,1,...,False,False,False,False,False,False,False,False,False,True
549598,25,28486,0,1,4443,4443,1482,1481,1480,0,...,False,False,False,False,False,False,False,False,False,True
5743966,277,11406,0,1,1768,1767,591,590,589,1,...,False,False,False,False,False,False,False,False,False,True
2408728,116,7095,0,1,1107,1107,371,370,369,1,...,False,False,False,False,False,False,False,False,False,True


In [5]:
error_list=['物料推送装置故障1001',
'物料检测装置故障2001',
'填装装置检测故障4001',
'填装装置定位故障4002',
'填装装置填装故障4003',
'加盖装置定位故障5001',
'加盖装置加盖故障5002',
'拧盖装置定位故障6001',
'拧盖装置拧盖故障6002'
]

In [6]:
# 将特定列中所有不等于 0 的值替换为 1
df['物料检测装置故障2001'] = (df['物料检测装置故障2001'] != 0).astype(int)

In [7]:
X=dfx
y=df['物料检测装置故障2001']
y.value_counts()

物料检测装置故障2001
1    23254
0    23254
Name: count, dtype: int64

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [9]:
# 特征缩放
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# 初始化随机森林分类器
model1 = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators是树的数量
model2 = SVC(kernel='rbf',class_weight='balanced',gamma=1,C=100)
model3 = XGBClassifier(random_state=120,
                  objective = 'multi:softmax',
                  eval_metric = ['mlogloss', 'aucpr', 'auc', 'map'],
                  num_class=2,
                  n_estimators=340,
                  reg_lambda=2,
                  min_child_weight=0,
                  max_depth=6,
                  learning_rate=0.1)
# 创建逻辑回归模型
model4 = LogisticRegression(max_iter=1500, penalty='l2', solver='lbfgs')


In [11]:
# 使用多数投票集成组合模型
ensemble = VotingClassifier(estimators=[
    ('RF', model1),
    ('SVC', model2),
    ('XGB', model3),
    ('LR',model4)
], voting='hard')

# # 在训练数据上拟合集成模型
# ensemble.fit(X_train, y_train)

# # 在测试数据上评估集成模型的性能
# print(f"整体精度：{ensemble.score(X_test, y_test) * 100:.2f}%")

# # 假设你有模型预测的标签和实际标签
# y_pred = ensemble.predict(X_test)
# conf_matrix = confusion_matrix(y_test, y_pred)

# print("混淆矩阵：")
# print(conf_matrix)

# # 打印分类报告，包括准确度、召回率和精确度
# print("\n分类报告：")
# print(classification_report(y_test, y_pred))

In [12]:
# 特征缩放
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [13]:
# 在训练数据上拟合集成模型
ensemble.fit(X, y)

VotingClassifier(estimators=[('RF', RandomForestClassifier(random_state=42)),
                             ('SVC',
                              SVC(C=100, class_weight='balanced', gamma=1)),
                             ('XGB',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=['mlogloss', 'aucpr',
                                                         'auc', '...
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None, min_child_weight=0,
                                            missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=340, n_jobs=None,
                                            num_class=2, num_parallel_tree=None,
                                            objective='multi:softmax', ...)),
                             ('LR', LogisticRegression(max_iter=1500))])

In [14]:
import joblib

# 保存训练好的集成模型到文件
joblib.dump(ensemble, "ensemble_model.pkl")

# 打印成功消息
print("训练好的集成模型已保存为 ensemble_model.pkl")


训练好的集成模型已保存为 ensemble_model.pkl
